In [29]:
from langchain_openai import ChatOpenAI
import os 
from dotenv import load_dotenv
load_dotenv()

True

In [30]:
from langchain.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

In [31]:
import xarray as xr

In [32]:
from arraylake import Client

In [33]:
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool, StructuredTool
from langchain_experimental.utilities import PythonREPL
from langchain.prompts import MessagesPlaceholder
from langchain.agents import AgentExecutor, create_tool_calling_agent

In [34]:
import uuid


In [62]:
indian = xr.open_zarr(
    "gcs://nmfs_odp_nwfsc/CB/mind_the_chl_gap/IO.zarr",
    storage_options={"token": "anon"},
    consolidated=True,        # source is consolidated
    chunks="auto",            # stream in chunks
)

In [66]:
indian.lat.max()

<xarray.DataArray 'lat' ()> Size: 4B
array(32., dtype=float32)

In [67]:
indian.lat.min()

<xarray.DataArray 'lat' ()> Size: 4B
array(-12., dtype=float32)

In [35]:
def read_arraylake_dataset(repo_name: str, group_name: str, variable: str = "all", branch: str = "main") -> xr.Dataset:
    
    client = Client()
    
    repo = client.get_repo(repo_name)
    
    session = repo.readonly_session(branch=branch)
    
    ds = xr.open_zarr(session.store, group=group_name, consolidated=False)

    if variable != "all":
        ds = ds[variable]

    path = f"llm_downloads/{uuid.uuid4()}"
    ds.to_zarr(path, "w")
    
    return path

#ds = read_arraylake_dataset(
#    "earthmover-public/era5-surface-aws",
#    "spatial"
#)

In [36]:
HF_TOKEN = os.environ["HF_TOKEN"]

In [37]:
llm = ChatOpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=HF_TOKEN,
    model="openai/gpt-oss-20b:fireworks-ai" 
)

In [38]:
python_repl = PythonREPL()

In [39]:
python_repl_tool = Tool(
    name="python_repl",
    func=python_repl.run,
    description=(
        "You are a python REPL. We will be tasking you to do "
        "different types of analyses on data. Do not mock any data. "
        "If there is no data to answer a question, say that you can not "
        "perform the task and don't run any code. Use matplotlib for plotting "
        "and pick appropriate color maps based on the variables in the query. "
        "You should also print an explanation of what you did when you return your "
        "analysis."
    ),
)

In [40]:
arraylake_tool = StructuredTool.from_function(
    func=read_arraylake_dataset,
    name="read_arraylake_dataset",
    description=(
        "Load an Arraylake dataset. Provide repo_name (e.g. 'earthmover-public/era5-surface-aws'), "
        "group_name (e.g. 'spatial'), optional variable (or 'all'), and branch (default 'main'). "
        "Returns path to downloaded dataset."
    ),
)

In [56]:
import s3fs
import xarray as xr
import re
def read_s3():
    s3 = s3fs.S3FileSystem(anon=True)

    ds = xr.open_zarr(
        store=s3fs.S3Map(
            root="s3://mur-sst/zarr-v1/",
            s3=s3,
            check=False
        )
    )

    ds_one_day = ds.sel(time="2020-01-01")

    path = f"llm_downloads/{uuid.uuid4()}.nc"
    ds_one_day.to_netcdf(path)
    return path



In [57]:
s3_tool = StructuredTool.from_function(
    func=read_s3,
    name="Read from s3",
    description="loads the mur dataset from s3. saves the file to a file and returns the path."
)

In [58]:
tools = [python_repl_tool, arraylake_tool, s3_tool]

In [59]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a data analyst that can answer general questions about data, as well as make plots and do some basic analysis using a python REPL tool. Use the path returned from the tools to read files for analysis."),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

In [60]:
agent = create_tool_calling_agent(
    llm=llm,
    tools=tools,
    prompt=prompt,
)

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [61]:
r = agent_executor.invoke({"input": "please plot me a variable of your choosing from the s3 dataset. just plot one hour's worth of data"})



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.



Invoking: `python_repl` with `import xarray as xr
import matplotlib.pyplot as plt

# Load dataset from S3
path = <...>`


SyntaxError('invalid syntax', ('<string>', 5, 8, 'path = <...>\n', 5, 9))
Invoking: `python_repl` with `# Step 1: Load dataset from s3
path = ""  # placeholder, will be replaced by tool output
`




APIError: server had an error while processing your request, please retry again after a brief wait